<a href="https://colab.research.google.com/github/kousiknandy/pycolab/blob/main/Wordcount_localfiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
# !git clone https://github.com/kousiknandy/pycolab.git

In [112]:
# !ls -R pycolab

In [113]:
import re
from collections import defaultdict

uninteresting = {'the','and','i','of','to','my','a','in','that','was','me',
    'with','but','had','you','he','which','it','as','not','his','for','by'}

def read_lines(filename):
    with open(filename, "r") as f:
        yield from f

def read_words(line):
    for l in line:
        for words in re.finditer(r"\b\w+\b", l, re.ASCII):
            if (w := words.group(0).lower()) not in uninteresting:
                yield w

def mapper(filename):
    wordcount = defaultdict(int)
    lines = read_lines(filename)
    words = read_words(lines)
    for w in words:
        wordcount[w] += 1
    return wordcount


In [114]:
def wordhash(word):
    h = 5381
    for c in word:
        h = h * 33 + ord(c)
    return h

def keypart():
    words = defaultdict(list)
    while w := (yield):
        words[w[0]].append(w[1])
    return words


In [115]:
def word_summation(words):
    try:
        words.send(None)
    except StopIteration as e:
        count_list = e.value
    res = {}
    for k,v in count_list.items():
        res[k] = sum(v)
    return res


In [116]:
from concurrent.futures import ThreadPoolExecutor
import glob

parts = 2
files = glob.iglob("pycolab/data/book/frankestine/*")
partitions = [keypart() for _ in range(parts)]
for i in range(parts):
    partitions[i].send(None)

with ThreadPoolExecutor(max_workers=4) as executor:
    res = executor.map(mapper, files)
for wordc in res:
    for w, c in wordc.items():
        partitions[wordhash(w) % parts].send((w,c))
with ThreadPoolExecutor(max_workers=2) as executor:
    res = executor.map(word_summation, partitions)
ans = {}
for wordc in res:
    ans = {**ans, **wordc}


In [117]:
import heapq
freq = [(v,k) for k,v in ans.items()]
topw = heapq.nlargest(25, freq)
[t[1] for t in topw]

['on',
 'this',
 'from',
 'be',
 'her',
 'have',
 'when',
 'is',
 'at',
 'were',
 'your',
 'she',
 'or',
 'him',
 'all',
 'an',
 'if',
 'they',
 'so',
 'one',
 'will',
 'could',
 'are',
 'been',
 'their']